In [ ]:
# load necessary libraries. install what you don't have
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time



## Testing out the MET Museum API


Refer to [metmuseum.github.io] (metmuseum.github.io) for guidelines on MET museum API search queries 

In [ ]:
'''
The following function takes data from a specific department in the MET museum. This can help with organizing your data, but is not required
a = department name
b = department number 
c = how many samples you want
'''
def get_image_data(a, b, c): #input is in department number and department name
    url_test = "https://collectionapi.metmuseum.org/public/collection/v1"
    departments = ("/search?departmentId=" + str(a) +"&hasImages=true&dateBegin=1800&dateEnd=2000&q=" + b)
    response = requests.get(url_test+departments)
    if response.status_code == 200 :
        print("API request successful!")
        data = response.json()
    else:
        print("Failed to retrieve data: {response.status_code}")

    print("Total Items Retrieved From " + b + " department: " + str(data['total']))
    
    img_url = []
    date_list = []
    domainList = []
    ID_list = data['objectIDs']
    
    index_val = 0 # if the distribution of years is wonky might want to consider random selection?
    counter_val = 0

    while counter_val <c: #set sample size
        object_id = ID_list[index_val]

        url_object = "https://collectionapi.metmuseum.org/public/collection/v1/objects/" 

        reponse_object = requests.get(url_object+str(object_id))

        if reponse_object.status_code == 200:
            data_ob = reponse_object.json()

            image_url = data_ob['primaryImage']
            if image_url != "" and data_ob.get("isPublicDomain"):
                img_url.append(image_url)
                date_val=data_ob['objectBeginDate']
                date_list.append(date_val)
                domainList.append(data_ob.get("isPublicDomain"))
            
                index_val+= 1
                counter_val+= 1
            else:
                index_val+=1
                counter_val=counter_val
            
            
            
            #conditional statement 

            #img_rep = requests.get(image_url).content
            #img = plt.imread(io.BytesIO(img_rep), format='JPG')
            #plt.imshow(img)

        # and this can be read directly as a np array i g? 
        else:
            #print("Failed to retrieve url")
            index_val+= 1
            counter_val= counter_val 
            

        
        time.sleep(0.05)
    df = {'Year': date_list,
        'URL': img_url,
        'isPublic' : domainList
        }
    df = pd.DataFrame(df)   
    df['Type'] = b
    return df
  

In [ ]:
# code to show an image/process image characteristics.
'''

img_rep = requests.get(image_url).content
    img = plt.imread(io.BytesIO(img_rep), format='JPG')
    plt.imshow(img)
'''

"\nimg_rep = requests.get(image_url).content\n    img = plt.imread(io.BytesIO(img_rep), format='JPG')\n    plt.imshow(img)\n"

In [ ]:
pt_url_list.head()


,Year,URL,isPublic,Type
0,1866,https://images.metmuseum.org/CRDImages/ep/orig...,True,painting
1,1804,https://images.metmuseum.org/CRDImages/ep/orig...,True,painting
2,1804,https://images.metmuseum.org/CRDImages/ep/orig...,True,painting
3,1846,https://images.metmuseum.org/CRDImages/ep/orig...,True,painting
4,1810,https://images.metmuseum.org/CRDImages/ep/orig...,True,painting


In [ ]:
from PIL import Image
from io import BytesIO
from tqdm import tqdm

# this code chunk lets you check the image dimensions of your dataset

def get_width(img):
    if img is not None:
        return img.width
    return None

def get_height(img):
    if img is not None:
        return img.height
    return None


def fetch_image_dimensions(df):
    widths = []
    heights = []

    for url in tqdm(df['URL'], desc="Fetching image dimensions"):
        time.sleep(0.05)
        try:
            response = requests.get(url, timeout=5)
            img = Image.open(BytesIO(response.content))
            widths.append(img.width)
            heights.append(img.height)
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")
            widths.append(None)
            heights.append(None)
    
    df = df.copy()
    df['width'] = widths
    df['height'] = heights
    return df


Fetching image dimensions: 100%|██████████| 10/10 [00:05<00:00,  1.86it/s]

Average Painting Size (10 samples): 2873.3 x 3670.7 pixels
Average Drawing Size (10 samples): 2602.0 x 3152.8 pixels
